In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, backend, constraints, initializers
from keras_self_attention import SeqSelfAttention
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('truncated_LOB_data_BTC_USD_COINBASE.csv')

In [ ]:
def finder_of_fulfilment(df, column):
    time, indicator = [], []
    for i in range(len(df)):
        num = df[column].iloc[i]
        arr = df[df[column]>num][column].index
        time.append(pd.to_datetime(df['timestamp'].iloc[arr[arr>i][0]]) - pd.to_datetime(df['timestamp'].iloc[i])
                    if len(arr[arr>i]) else
                    pd.to_datetime('2023-10-03 00:00:00') - pd.to_datetime(df['timestamp'].iloc[i]))
        indicator.append(1 if len(arr[arr>i]) else 0)
    return time, indicator

In [ ]:
df['time'], df['indicator'] = finder_of_fulfilment(df, 'ask_prices_0')
df

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['time'] = (df['time']).dt.total_seconds().astype(int)
df = df.iloc[0:-140]

In [ ]:
df_to_plot = df[['indicator', 'time']]
N = len(df_to_plot)
S_of_t = []

for time in range(max(df['time'])):
    num_executed = len(df_to_plot[df_to_plot['time']==time])
    num_survived = len(df_to_plot[df_to_plot['time']>time])
    prob = num_executed/(num_survived+num_executed)
    #print(prob)
    S_of_t.append(1-prob)

In [ ]:
S_hat = np.cumprod(S_of_t)

In [ ]:
t = np.linspace(0, max(df['time']), max(df['time']))

plt.figure(figsize=(6, 6))
plt.plot(t[:400], S_hat[:400], linewidth=2, label='Level 1, pegged', color='black')
plt.xlabel('t (sec.)')
plt.ylabel('$\hat{S}(t)$')
plt.title('Survival Probability Over Time')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
#Make some features and drop useless columns
pd.options.mode.chained_assignment = None

df['vol_imbalance'] = (df['bid_quantity_0'] - df['ask_quantity_0'])/(df['bid_quantity_0'] + df['ask_quantity_0'])
df['microprice'] = ((df['bid_prices_0']*df['bid_quantity_0']+df['ask_prices_0']*df['ask_quantity_0'])
                    /(df['bid_quantity_0'] + df['ask_quantity_0']))

In [ ]:
target = [] #create target S(execution_time)
for i in range(len(df)):
    target.append(S_of_t[df['time'].iloc[i]-1])

df['target'] = target

In [ ]:
plt.figure(figsize=(10, 6))

# Plotting the microprice with a blue line
plt.plot(df['timestamp'], df['microprice'], color='blue', linewidth=2, label='Microprice')

# Adding labels and title
plt.xlabel('Time')
plt.ylabel('Microprice')
plt.title('Microprice over Time')

# Displaying the grid
plt.grid(True)

In [ ]:
plt.figure(figsize=(10, 6))

# Plotting the microprice with a blue line
plt.plot(df['timestamp'], df['vol_imbalance'].rolling(1000).mean(), color='blue',
         linewidth=2, label='Rolling Volume Imbalance')

# Adding labels and title
plt.xlabel('Time')
plt.ylabel('Volume Imbalance')
plt.title('Rolling Volume Imbalance over Time')

# Displaying the grid
plt.grid(True)

In [ ]:
plt.figure(figsize=(10, 6))

# Plotting the microprice with a blue line
plt.plot(df['timestamp'], df['target'].rolling(1000).mean(), color='blue',
         linewidth=2, label='Target thing')

# Adding labels and title
plt.xlabel('Time')
plt.ylabel('Target')
plt.title('Rolling TargetTargetTarget')

# Displaying the grid
plt.grid(True)

In [ ]:
#df.set_index('timestamp', drop=True, inplace=True)
scaler = StandardScaler()
X = df.drop(columns=['timestamp', 'time', 'indicator', 'target'])
Y = df['target'].iloc[500:].values

In [ ]:
tensor_slices = []

for i in range(500, len(X)):
    # Slice the last 100 rows, including the current row
    start_index = max(0, i - 500)  # Adjust index to ensure at least 100 rows
    slice_df = X.iloc[start_index:i+1]

    # Convert the slice to a tensor and append it to the list
    tensor_slice = tf.convert_to_tensor(slice_df, dtype=tf.float32)
    tensor_slices.append(tensor_slice)

# Stack the slices to create a 3D tensor
X = tf.stack(tensor_slices)

print(X.shape)

In [ ]:
Y.shape

# Simplified Staff (LSTM instead of Att)

input_shape = (100, 22)  # Assuming X is your data
steps = 1  # You need to determine the appropriate number of steps based on your data
encoder_input = layers.Input(shape=input_shape)
encoder_reshape = layers.Reshape(new_input_shape)(encoder_input)
encoder_conv1 = layers.Conv1D(32, kernel_size=2, activation='relu')(encoder_reshape)
encoder_lstm = layers.LSTM(32, activation='relu')(encoder_conv1)
latent_dim = 32
encoder_output = layers.Dense(latent_dim)(encoder_lstm)

encoder_model = Model(encoder_input, encoder_output)

# Define the decoder
decoder_input = layers.Input(shape=(latent_dim,))
decoder_dense1 = layers.Dense(32, activation='relu')(decoder_input)
decoder_reshape = layers.Reshape((1, 32))(decoder_dense1)
decoder_conv1 = layers.Conv1D(32, kernel_size=3, activation='relu', padding='same')(decoder_reshape)
output_dim = 32
decoder_output = layers.Dense(output_dim, activation='sigmoid')(decoder_conv1)

decoder_model = Model(decoder_input, decoder_output)

# Combine the encoder and decoder into an autoencoder
autoencoder_input = layers.Input(shape=input_shape)
encoded = encoder_model(autoencoder_input)
decoded = decoder_model(encoded)
autoencoder_model = Model(autoencoder_input, decoded)

# Compile the model
autoencoder_model.compile(optimizer='adam', loss='MAE')

# Train the model
autoencoder_model.fit(X, Y, epochs=3, batch_size=32)

# Some experiements (model is almost ready monotonicity change is needed or S(t) to f(t) transition)

In [ ]:
plt.plot(Y)

In [ ]:
# Define the DCC layer
class DilatedCausalConvolution(layers.Layer):
    def __init__(self, filters, kernel_size, dilation_rate):
        super().__init__()
        self.query_conv = layers.Conv1D(filters=filters, kernel_size=kernel_size,
                                        dilation_rate=dilation_rate, padding='same', activation='sigmoid')
        self.key_conv = layers.Conv1D(filters=filters, kernel_size=kernel_size,
                                      dilation_rate=dilation_rate, padding='same', activation='sigmoid')
        self.value_conv = layers.Conv1D(filters=filters, kernel_size=kernel_size,
                                        dilation_rate=dilation_rate, padding='same', activation='sigmoid')

    def call(self, inputs):
        query = self.query_conv(inputs)
        key = self.key_conv(inputs)
        value = self.value_conv(inputs)
        return query, key, value

# Define the Transformer block
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, d_model):
        super().__init__()
        self.multi_head_attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)

    def call(self, query, key, value):
        attn_output = self.multi_head_attention(query, key, value)
        proj_output = tf.concat(attn_output, axis=0)
        return proj_output

# Define the encoder using the DCC and Transformer block
def create_encoder(input_shape, filters, kernel_size, dilation_rate, num_heads, d_model):
    inputs = layers.Input(shape=input_shape)
    dcc_layer = DilatedCausalConvolution(filters=filters, kernel_size=kernel_size, dilation_rate=dilation_rate)
    query, key, value = dcc_layer(inputs)
    transformer_block = TransformerBlock(num_heads=num_heads, d_model=d_model)
    transformer_output = transformer_block(query, key, value)
    model = models.Model(inputs=inputs, outputs=transformer_output)
    return model

# Define the monotonic decoder
def monotonic_constraint(weight_matrix):
    return tf.where(weight_matrix < 0, tf.zeros_like(weight_matrix), weight_matrix)

class CustomDense(layers.Layer):
    def __init__(self, units, activation='sigmoid', **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        last_dim = input_shape[-1] if isinstance(input_shape[-1], int) else input_shape[-1].value
        self.kernel = self.add_weight(name='kernel', shape=(last_dim, self.units),
                                      initializer='zeros', constraint=monotonic_constraint, trainable=True)
        self.bias = self.add_weight(name='bias', shape=(self.units,), initializer='zeros', trainable=False)
        super().build(input_shape)

    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.kernel) + self.bias)

class CustomDecoder(layers.Layer):
    def __init__(self, input_dim, units, activation='sigmoid', num_layers=5, **kwargs):
        super().__init__(**kwargs)
        self.num_layers = num_layers
        self.dense_layers = [CustomDense(units, activation=activation) for _ in range(num_layers)]
        self.input_dim = input_dim

    def build(self, input_shape):
        self.dense_layers[0].build((None, self.input_dim))

    def call(self, inputs):
        x = inputs
        for layer in self.dense_layers:
            x = layer(x)
        return x

# Combine the encoder and decoder to create the full model
def create_full_model(input_shape, filters, kernel_size, dilation_rate, num_heads, d_model, output_shape, units):
    encoder_inputs = layers.Input(shape=input_shape)
    encoder = create_encoder(input_shape, filters, kernel_size, dilation_rate, num_heads, d_model)
    encoder_output = encoder(encoder_inputs)

    encoder_output_dim = encoder_output.shape[-1]

    decoder = CustomDecoder(input_dim=encoder_output_dim, units=units, activation='sigmoid')
    decoder_output = decoder(encoder_output)
    flat_output = tf.keras.layers.Flatten()(decoder_output)
    final_output = tf.keras.layers.Dense(output_shape)(flat_output)
    full_model = models.Model(inputs=encoder_inputs, outputs=final_output)

    return full_model

In [ ]:
# Define model parameters
input_shape = (501, 22)  # Adjust based on your actual data dimensions
filters = 8
units = 32
kernel_size = 8
dilation_rate = 2
num_heads = 3
d_model = 64
output_shape = 1

In [ ]:
# Create the model
model = create_full_model(input_shape, filters, kernel_size, dilation_rate, num_heads, d_model, output_shape, units)

# Create TensorFlow dataset
train_ds = tf.data.Dataset.from_tensor_slices((X, Y)).batch(32)

# Compile the model
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=1000, decay_rate=0.96, staircase=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=tf.keras.losses.MeanAbsoluteError())

# Train the model
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15)
model.fit(train_ds, epochs=10, callbacks=[callback])

In [ ]:
Y_pred = model.predict(X)

Y_pred.mean()

In [ ]:
plt.plot(Y_pred)

In [ ]:
Y

In [ ]:
model_att.add(SeqSelfAttention(units=1, attention_activation='sigmoid', attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, input_shape=(X.shape[1], 1)))
model_att.add(layers.Flatten())
model_att.add(layers.Dense(1, activation='linear'))
model_att.compile(loss='mae', optimizer='adam')
history = model_att.fit(X, Y, epochs=100, verbose=2)

In [ ]:
SeqSelfAttention.